In [5]:
import sys
!{sys.executable} -m pip install 'gensim'
!{sys.executable} -m pip install 'nltk'
!{sys.executable} -m pip install 'pyLDAvis'

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.3/23.3 MB 21.1 MB/s eta 0:00:00m eta 0:00:010:01:01
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 5.6 MB/s eta 0:00:00
  Using cached scipy-1.12.0-cp312-cp312-macosx_12_0_arm64.whl.metadata (217 kB)
  Using cached smart_open-7.0.4-py3-none-any.whl.metadata (23 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.7/13.7 MB 26.0 MB/s eta 0:00:00m eta 0:00:010:0101
Using cached scipy-1.12.0-cp312-cp312-macosx_12_0_arm64.whl (31.4 MB)
Using cached smart_open-7.0.4-py3-none-any.whl (61 kB)
  Created wheel for gensim: filename=gensim-4.3.2-cp312-cp312-macosx_14_0_arm64.whl size=23965325 sha256=007bca22d88ca0c8bd50993380a02eb99b89f72cd7417ef5c512b2e3c75d4bf4
  Stored in directory: /Users/400649/Library/Caches/pip/wheels/50/c0/ac/7bb08954bc59d390c848b480a3fc5eec68c14bc77bf334d624
Successfully built gen

In [7]:
import gensim
from gensim import corpora
from pprint import pprint
import pandas as pd
import nltk
from nltk.tokenize import word_tokenize
import pyLDAvis.gensim_models
import string
import re
nltk.download('punkt')

[nltk_data] Downloading package punkt to /Users/400649/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [41]:
# aineiston lataus (VARMISTA SIJAINTI)
file_path = "ainsto283.csv"
data = pd.read_csv(file_path, sep=';', dtype={'puhe': str})
data = data.iloc[:, :3]
data = data.dropna(subset=['puhe'])

KeyError: ['puhe']

In [42]:
# välimerkkien ja numeroiden poisto
def remove_punctuation_and_numbers(text):
    text = text.translate(str.maketrans('', '', string.punctuation))
    text = re.sub(r'\d+', '', text)
    return text

In [43]:
data['puhe'] = data['puhe'].apply(remove_punctuation_and_numbers).str.lower()

KeyError: 'puhe'

In [44]:
data.head()

,id,nimi,sukupuoli
0,1,"Aalto, Touko",m
1,2,"Aalto, Touko",m
2,3,"Aalto, Touko",m
3,4,"Aalto, Touko",m
4,5,"Aalto, Touko",m


In [45]:
# dokumenttien tokenisointi
documents = data['puhe'].tolist()
tokenized_documents = [doc.lower().split() if isinstance(doc, str) else [] for doc in documents]

KeyError: 'puhe'

In [46]:
# stoplistan lataus (VARMISTA SIJAINTI)
stopword_file_path = "finnish.txt"
with open(stopword_file_path, "r", encoding="utf-8") as file:
    finnish_stopwords = set(file.read().splitlines())

In [47]:
# stop-sanojen poisto
filtered_documents = [[word for word in doc if word.lower() not in finnish_stopwords] for doc in tokenized_documents]
filtered_documents_strings = [' '.join(doc) for doc in filtered_documents]
data['puhe'] = filtered_documents_strings

In [48]:
data['puhe']

0       historian toistunut kuvio elämän alueelta hävi...
1       jätän elon epäasialliset puheet arvoonsa kysyn...
2       suomen menestystarina kertomus koulutuksen voi...
3       hyvän talous työllisyyskehityksen kääntöpuolen...
4       salissa puhutaan velasta pääministerikin puhui...
                              ...                        
1410    aito avioliitto ‑kansalaisaloite keräsi nimeä ...
1411    hakkaraiselta herunut ymmärrystä suomenruotsal...
1412    puheenvuoroja kertaalleen lupaukset kirkko saa...
1413    keskustelusta käy selvästi ilmi ihmiset vieläk...
1414    ajattelin lukea eräältä kansalaiselta tulleen ...
Name: puhe, Length: 1415, dtype: object

In [49]:
# sanakirjan ja korpuksen luonti
dictionary = corpora.Dictionary(filtered_documents)
corpus = [dictionary.doc2bow(doc) for doc in filtered_documents]

In [94]:
# LDA-mallin koulutus (VALITSE AIHEIDEN JA ITERAATIOIDEN MÄÄRÄ)
lda_model = gensim.models.LdaModel(corpus, num_topics=50, id2word=dictionary, passes=25)

In [95]:
# tulosten visualisointi
lda_display = pyLDAvis.gensim_models.prepare(lda_model, corpus, dictionary, sort_topics=False)
pyLDAvis.display(lda_display)

/opt/homebrew/Cellar/jupyterlab/4.1.5/libexec/lib/python3.12/site-packages/joblib/_utils.py:38: DeprecationWarning: ast.Num is deprecated and will be removed in Python 3.14; use ast.Constant instead
  if isinstance(node, ast.Num):  # <number>
/opt/homebrew/Cellar/jupyterlab/4.1.5/libexec/lib/python3.12/site-packages/joblib/_utils.py:38: DeprecationWarning: ast.Num is deprecated and will be removed in Python 3.14; use ast.Constant instead
  if isinstance(node, ast.Num):  # <number>
/opt/homebrew/Cellar/jupyterlab/4.1.5/libexec/lib/python3.12/site-packages/joblib/_utils.py:39: DeprecationWarning: Attribute n is deprecated and will be removed in Python 3.14; use value instead
  return node.n
/opt/homebrew/Cellar/jupyterlab/4.1.5/libexec/lib/python3.12/site-packages/joblib/_utils.py:38: DeprecationWarning: ast.Num is deprecated and will be removed in Python 3.14; use ast.Constant instead
  if isinstance(node, ast.Num):  # <number>
/opt/homebrew/Cellar/jupyterlab/4.1.5/libexec/lib/python3.1

In [96]:
lda_model.get_document_topics(corpus)

In [99]:
document_topics = lda_model.get_document_topics(corpus)

In [77]:
[print(topic) for topic in document_topics]

[(166, 0.9849211)]
[(70, 0.98869145)]
[(7, 0.012583263), (17, 0.022138676), (19, 0.011134021), (30, 0.011473), (34, 0.059078425), (37, 0.01800365), (46, 0.0138955815), (69, 0.1265374), (88, 0.015517332), (92, 0.016162332), (97, 0.022240508), (104, 0.017168108), (106, 0.011113365), (118, 0.02595063), (121, 0.013781742), (135, 0.015484826), (154, 0.049896207), (157, 0.038746476), (158, 0.020722367), (181, 0.01574459), (183, 0.12077074), (189, 0.07982616), (194, 0.034695107), (197, 0.01079266), (198, 0.030002473), (199, 0.10237568)]
[(6, 0.013425716), (34, 0.021046543), (36, 0.015610812), (46, 0.09311534), (49, 0.040336996), (97, 0.08642105), (107, 0.04785778), (109, 0.018553445), (113, 0.010069733), (118, 0.07050527), (142, 0.13310772), (150, 0.4364684)]
[(69, 0.9937032)]
[(44, 0.01909151), (46, 0.060785726), (49, 0.013731387), (107, 0.025996407), (118, 0.021120878), (166, 0.052099034), (183, 0.791102)]
[(11, 0.038877647), (34, 0.10984403), (115, 0.78248477), (157, 0.052979786)]
[(39, 0.

[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,

In [100]:
topic_assignments = pd.DataFrame({
    'ID': data['id'],
    'Topic': [topic[0][0] for topic in document_topics],
    'Topic_Prob': [topic[0][1] for topic in document_topics],
    'Gender': data['sukupuoli']
})

In [101]:
document_topic_df = pd.DataFrame(document_topics)

In [102]:
merged_data = pd.merge(topic_assignments, document_topic_df, left_index=True, right_index=True)

In [108]:
topic_ids = merged_data[merged_data['Topic'] == 10]

In [109]:
topic_sorted_ids = topic_ids.sort_values(by=0, ascending=False)

In [110]:
topic_sorted_ids

,ID,Topic,Topic_Prob,Gender,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
752,753,10,0.995999,f,"(10, 0.9959986)",None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
576,577,10,0.993911,m,"(10, 0.99391085)",None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
465,466,10,0.987432,m,"(10, 0.9874317)",None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
1343,1344,10,0.985995,f,"(10, 0.985995)",None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
985,986,10,0.984917,f,"(10, 0.98491734)",None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
1339,1340,10,0.983384,f,"(10, 0.98338425)",None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
189,190,10,0.980393,f,"(10, 0.9803928)",None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
1008,1009,10,0.819069,f,"(10, 0.81904066)","(33, 0.17713176)",None,None,None,None,None,None,None,None,None,None,None,None,None,None
334,335,10,0.729706,m,"(10, 0.7296898)","(21, 0.25147694)",None,None,None,None,None,None,None,None,None,None,None,None,None,None
663,664,10,0.627127,m,"(10, 0.6271288)","(29, 0.015810594)","(33, 0.3538488)",None,None,None,None,None,None,None,None,None,None,None,None,None


In [111]:
gender_counts = topic_sorted_ids['Gender'].value_counts()

In [112]:
gender_counts

Gender
f    6
m    4
Name: count, dtype: int64